In [0]:
#importing of necessary libraries and modules (recommended to run in Google colab)

import numpy as np
import matplotlib.pyplot as plt
import pickle
from matplotlib import rc
rc('font',**{'family':'sans-serif','sans-serif':['Helvetica'], 'size':24})
rc('xtick', labelsize=24)
rc('ytick', labelsize=24)

import tensorflow as tf
import tensorflow_probability as tfp
import scipy.linalg as lg

from google.colab import drive
drive.mount('/content/gdrive')

'''we need to import tf_mpa,
thus, we go to the GAN_for_quantum_data
directory where the file is situated'''
%cd gdrive
%cd My Drive
%cd GAN_for_quantum_data

import tf_mpa

### Here we define all necessary building blocks for further mps

In [0]:
#"direction" of the hamiltonian
X = np.array([0.3, 1.1, 0.6])

#hamiltonian
H = np.einsum('i,ijk,ilm->jlkm' ,X, tf_mpa.sigma, tf_mpa.sigma).reshape((4, 4))

#time of collision
dt = 0.3

#unitary matrix
U = lg.expm(-1j * dt * H).reshape((2, 2, 2, 2))

#initial states (wave functions)
psi_in = np.array([1, 0], dtype=np.complex64)
psi_coll = np.array([0, 1], dtype=np.complex64)

#update tensors (building blocks of update mps)
mid_update_tensor = U.transpose((0, 1, 3, 2))
right_update_tensor = np.einsum('ijkl,l->ijk', mid_update_tensor, psi_coll).reshape((2, 2, 2, 1))

### Here we calculate dens. matrix of a chain after collisons

In [0]:
tf.reset_default_graph()

#number of collisions
num_of_colisions = 4

#number of spins
num_of_spins = 32

#initial state of chain
mpa_in_psi = tf_mpa.mpa([psi_in.reshape((1,) + psi_in.shape + (1,))] * num_of_spins)

#update tensor
mpa_update = tf_mpa.mpa([mid_update_tensor] * (num_of_spins - 1) + [right_update_tensor])

#collisions loop
for i in range(num_of_colisions):
    mpa_in_psi = tf_mpa.mpa.einsum('ji,i->j', mpa_update, mpa_in_psi)

#mpa dens. matrix
mpa_in_psi_conj = mpa_in_psi.copy()
mpa_in_psi_conj.conj()
mpa_rho = tf_mpa.mpa.einsum('i,j->ij', mpa_in_psi, mpa_in_psi_conj)

#evaluate dens matrix
sess = tf.Session()
rho_set_of_tensors = mpa_rho.eval_local_tensors(sess)

#taking trace over extra particle
rho_set_of_tensors[0] = np.expand_dims(np.einsum('iiklm->klm', \
rho_set_of_tensors[0].reshape((2 ** num_of_colisions, 2 ** num_of_colisions, 2, 2, -1))), axis=0)

### Here we generate and save dataset (measurement outcomes) and save mpa object which contains information about all correlation functions

In [0]:
#local observables
local_obs = np.append(np.expand_dims(tf_mpa.idm, axis=0), tf_mpa.sigma, axis=0)
local_obs = local_obs.reshape((1,) + local_obs.shape + (1,))

tf.reset_default_graph()

#wrap up local tensors of rho into mpa object
mpa_rho = tf_mpa.mpa(rho_set_of_tensors)

#wrap up local tensors of observables into mpa object
local_obs_mpa = tf_mpa.mpa([local_obs] * num_of_spins)

sess = tf.Session()

#this set of tensors contains information about all correlation functions
corr = tf_mpa.mpa.einsum('kij,ji->k', local_obs_mpa, mpa_rho).eval_local_tensors(sess)

#saving set of tensors
file_handler = open("collision model data/corr_functions.obj", 'wb')
pickle.dump(corr, file_handler)

import time
start = time.time()

#sample 10^6 samples
samples = np.array([mpa_rho.sample(100000, sess) for _ in range(10)]).reshape((1000000, -1))
print(time.time() - start)

#saving
np.save("collision model data/samples.npy", samples)